To train this agent, click _Runtime_ and press _Run all_. Make sure you've enabled a free Tesla T4 GPU!

<div class="align-center">
<a href="https://github.com/openpipe/art"><img src="https://github.com/openpipe/art/raw/main/assets/ART_pill.png" height="50"></a>
<a href="https://discord.com/invite/dnseNZuQ"><img src="https://github.com/openpipe/art/raw/main/assets/Discord_pill.png" height="50"></a>
<a href="https://openpipe.ai/blog/art-e-mail-agent"><img src="https://github.com/openpipe/art/raw/main/assets/ART_E_pill.png" height="50"></a>

Questions? Join the Discord and ask away! For feature requests or to leave a star, visit our [GitHub](https://github.com/openpipe/art).

</div>

<a href="https://art.openpipe.ai/"><img src="https://github.com/openpipe/art/raw/main/assets/Header_separator.png" height="5"></a>

This notebook shows how to train a Qwen 2.5 3B model to play Codenames. It will demonstrate how to set up a multi-turn agent, how to train it, and how to evaluate it.

Completions will be logged to OpenPipe, and metrics will be logged to Weights & Biases.

You will learn how to construct an [agentic environment](#Environment), how to define a [rollout](#Rollout), and how to run a [training loop](#Loop).


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

import psutil

ram_gb = psutil.virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

### Installation


In [ ]:
%%capture
!uv pip install openpipe-art==0.3.11.post3 "gql<4" --prerelease allow --no-cache-dir

In [ ]:
!uv pip install openpipe

### Environment Variables

Later on in the notebook, we'll be creating a model that can automatically logs metrics to Weights & Biases. In order to do so, you'll need to provide your Weights & Biases API key as an environment variable.

You can also optionally initiate an OpenPipe client to report completions to a [dashboard](https://app.openpipe.ai) to get a feel for what the completions your model is generating look like, and how they change over time. Logging to OpenPipe is free, but is not required for training!


In [ ]:
import os

# Optional - We need an OPENAI_API_KEY to run validation. This is optional, and we will skip validation if not provided
OPENAI_API_KEY=""
if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Optional
WANDB_API_KEY=""
if WANDB_API_KEY:
    os.environ["WANDB_API_KEY"] = WANDB_API_KEY

# Optional
OPENPIPE_API_KEY = ""
if OPENPIPE_API_KEY:
    os.environ["OPENPIPE_API_KEY"] = OPENPIPE_API_KEY

# Agentic Environment

<a name="Environment"></a>

ART allows your agent to learn by interacting with its environment. In this example, we'll create an environment in which the agent can play Codenames.

Feel free to read as much or as little of this section's code as you'd like. The important thing to understand is that we're defining the rules of this agent's environment. In many cases, this will already be defined by the task you're trying to solve, but if you need to define a custom environment, this is how you do it.


In [ ]:
# Download the 400 original codenames words and a dictionary to ensure that the clue is not outside of this dictionary
import requests

codenames_words_path = "https://raw.githubusercontent.com/OpenPipe/ART/refs/heads/main/examples/codenames/codenames_words.json"
dictionary_path = "https://raw.githubusercontent.com/OpenPipe/ART/refs/heads/main/examples/codenames/dictionary.json"
CODENAMES_WORDS = requests.get(codenames_words_path).json()
DICTIONARY = requests.get(dictionary_path).json()

In [ ]:
# Basic imports to get us running
# ───────────────────────────────  Imports  ────────────────────────────────
import asyncio
import copy
import random
import re
import time
import uuid
from typing import Dict, List, Tuple

from tqdm.asyncio import tqdm_asyncio

import art
from openpipe.client import AsyncOpenPipe

In [ ]:
# ───────────────────────────────  Config  ──────────────────────────────
NUM_STEPS = 100
NUM_GAMES_PER_STEP = 10
NUM_ROLLOUTS_PER_GAME = 20
# reduce all three above to 5 to run the example quickly

MODEL_NAME = "codenames_model_op_5"
BASE_MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"
VAL_BENCHMARK_MODEL = "gpt-4.1-nano"
NUM_VALIDATION_GAMES = 50
LEARNING_RATE = 5e-6
BETA = 0.0
DEBUG = False

# — reward constants —
# The following is the weightage for different kinds of outcomes from a game, to determine the reward
MARGIN_REWARD_DICT = {
    "turn_limit": -100,  # if the game reaches the turn limit for some reason, the team gets this penalty
    "error": -100,  # if the game ends in an error, the team player that caused the error gets this penalty (e.g. spymaster gets this penalty if the clue is invalid)
    "assassin": -30,  # if the team guesses the assassin, they get this penalty (both spymaster and operative get the penalty, since it could be an issue with the spymaster's clue or the operative's guess)
    "num_words_multiplier": 10,  # the team gets this reward for each word they guess (fraction of words they guess correctly * num_words_multiplier)
    "margin_multiplier": 1,  # the team gets this reward for however many words they defeat the other team by (difference in the number of words * margin_multiplier) (the losing team gets the negative of this)
}

In [ ]:
from openai import AsyncOpenAI


# ────────────────────────  1. Utility helpers  ────────────────────────────
async def call_llm(
    model: art.Model,
    messages: List[Dict[str, str]],
    temperature: float = 0.7,
    max_tokens: int = 500,
    num_retries: int = 3,
):
    """
    Minimal, retry‑aware wrapper around `chat.completions.create`.
    """
    for attempt in range(num_retries):
        try:
            client = AsyncOpenAI(
                base_url=model.inference_base_url,
                api_key=model.inference_api_key,
            )
            resp = await client.chat.completions.create(
                model=model.name,
                messages=messages,
                temperature=temperature,
                max_tokens=max_tokens,
                logprobs=True,
                store=False,
            )
            if not resp:
                raise RuntimeError("empty response")
            return resp
        except Exception as e:
            print(f"LLM error ({attempt + 1}/{num_retries}): {e}")
            await asyncio.sleep(1)
    raise RuntimeError("LLM call failed after retries")


# ───────────────────────  2. Core game classes  ───────────────────────────
class Card:
    """
    One word on the Codenames board.
    """

    def __init__(self, word: str, color: str):
        self.word = word
        self.color = color  # blue | red | neutral | assassin
        self.revealed = False

    def __repr__(self) -> str:
        return f"{self.word}{'(X)' if self.revealed else ''}:{self.color}"


class GameState:
    """
    Full mutable game state with an event log.
    """

    def __init__(
        self,
        cards: List[Card],
        starting_team: str = "blue",
        game_id: int = 0,
    ):
        self.cards = cards
        self.current_team = starting_team
        self.words_left = {
            "blue": sum(1 for c in cards if c.color == "blue"),
            "red": sum(1 for c in cards if c.color == "red"),
        }
        self.starting_team = starting_team
        self.game_id = game_id
        self.turn_count = 0

        self.last_event_indices: Dict[str, int] = {
            "blue_spymaster": -1,
            "blue_operative": -1,
            "red_spymaster": -1,
            "red_operative": -1,
            "game_engine": -1,
        }
        self.game_events: List[Dict[str, str]] = []

        self.game_ended_in_error: Dict[str, str] | None = None
        self.game_ended_in_assassin = False
        self.game_ended_in_turn_limit = False

        self.all_words = DICTIONARY

    # ───────── event log helpers ─────────
    def log_event(self, role_key: str, description: str) -> None:
        self.game_events.append({role_key: description})
        self.last_event_indices[role_key] = len(self.game_events) - 1
        if DEBUG:
            print(f"{role_key}: {description}")

    def format_events_for_player(self, role_key: str) -> str:
        last_seen = self.last_event_indices.get(role_key, -1)
        new_events = self.game_events[last_seen + 1 :]
        if not new_events:
            return ""
        lines = ["--- events since your last turn ---"]
        for ev in new_events:
            for rk, desc in ev.items():
                lines.append(f"{rk}: {desc}")
        lines.append("----------------------------------")
        return "\n".join(lines)

    # ───────── board helpers ─────────
    def reveal_card(self, card: Card) -> str | None:
        if card.revealed:
            return None
        card.revealed = True
        if card.color in ("blue", "red"):
            self.words_left[card.color] -= 1
        return card.color

    def get_unrevealed_cards(self) -> List[Card]:
        return [c for c in self.cards if not c.revealed]

    def get_revealed_cards(self) -> List[Card]:
        return [c for c in self.cards if c.revealed]

    # ───────── rules / win conditions ─────────
    def check_end_game(self) -> Tuple[str | None, str | None]:
        # assassin?
        if any(c for c in self.cards if c.color == "assassin" and c.revealed):
            loser = self.current_team
            winner = "red" if loser == "blue" else "blue"
            self.game_ended_in_assassin = True
            return winner, f"{loser} revealed the assassin"
        # all words guessed?
        if self.words_left["blue"] == 0:
            return "blue", "Blue found all their words"
        if self.words_left["red"] == 0:
            return "red", "Red found all their words"
        return None, None

    def switch_turn(self) -> None:
        self.current_team = "red" if self.current_team == "blue" else "blue"

    def follows_clue_rules(self, clue_word: str) -> bool:
        # single token
        if len(clue_word.split()) > 1:
            return False
        # letters / hyphens only
        if re.search(r"[^A-Za-z\-]", clue_word):
            return False
        # cannot be an unrevealed board word
        if clue_word.lower() in {c.word.lower() for c in self.get_unrevealed_cards()}:
            return False
        # must be in dictionary
        if clue_word.lower() not in self.all_words:
            return False
        return True


def create_operative_feedback(max_guesses, guesses_taken, guess, color, outcome):
    operative_feedback = ""
    if outcome.startswith("correct"):
        operative_feedback = (
            f"Your guess '{guess}' was correct! It belongs to your team ({color})."
        )
        if guesses_taken >= max_guesses:
            operative_feedback += f" Max guesses reached ({max_guesses}), ending turn."
        else:
            operative_feedback += " Either provide your next guess from the unrevealed cards on the board, or say end_turn."
    elif outcome == "neutral":
        operative_feedback = f"Your guess '{guess}' was neutral. Your turn ends here."
    elif outcome.startswith("other_team"):
        operative_feedback = f"Your guess '{guess}' belongs to the other team ({color}). Your turn ends here."
    return operative_feedback


# ───────────────────────  3. Game orchestration  ──────────────────────────
def create_game(
    num_words: int = 25,
    blue_count: int = 9,
    red_count: int = 8,
    assassin_count: int = 1,
    neutral_count: int = 7,
    starting_team: str = "blue",
    game_id: int = 0,
) -> GameState:
    all_codenames_words = CODENAMES_WORDS
    random.shuffle(all_codenames_words)
    codenames_words = all_codenames_words[:num_words]

    colors = (
        ["blue"] * blue_count
        + ["red"] * red_count
        + ["assassin"] * assassin_count
        + ["neutral"] * neutral_count
    )
    random.shuffle(colors)

    cards = [Card(w, c) for w, c in zip(codenames_words, colors)]
    return GameState(cards, starting_team, game_id)

In [ ]:
# ───────────────────────  LLM player classes  ──────────────────────────
SPYMASTER_SYSTEM_PROMPT = """You are a SPYMASTER in a game of Codenames.
You are on a team (blue or red).
You see which words belong to blue, red, neutral, and assassin.
Your objective:
- Give a ONE-WORD CLUE and a NUMBER, to guide your operative to guess your team's words.
- Avoid giving a clue that makes your operative guess the assassin word, or else you lose immediately.
- If the operative guesses a neutral or enemy word, the word is revealed and your turn ends, but the game continues.
Strict format rules:
- You must propose exactly one single-word clue
- The clue must not be a word on the board
- Your output must be in the format: clue_word, number (e.g. "space, 2")
If you ever deviate from that exact output format, you lose.
Please follow these instructions exactly."""

SPYMASTER_USER_PROMPT = """Your turn. You are the {team} spymaster.
Here is the current board (from your perspective), showing:
  - which words are unrevealed (and their colors),
  - which words are revealed (and their colors).
You need to give a single-word clue and a number of words you want your operative to guess.

{board}

Please provide your ONE-WORD clue and a number, in the format: clue_word, number"""

OPERATIVE_SYSTEM_PROMPT = """You are an OPERATIVE in a game of Codenames.
- Your spymaster gives you a ONE-WORD CLUE and a NUMBER.
- You must guess words from the unrevealed board that match that clue.
- You propose exactly ONE guess at a time, or say "end_turn" to stop guessing.
- If you guess an enemy word or neutral word, your turn ends.
- If you guess the assassin word, you immediately lose.
- The clue's NUMBER is how many words the spymaster believes are related, but you can guess up to NUMBER + 1 (or fewer).
Format rules:
- Output must be either a single unrevealed word present on the board or "end_turn".
- If you deviate from that, you lose."""

OPERATIVE_USER_PROMPT = """Your turn. You are the {team} operative.
Your spymaster's clue is: "{clue}" (number: {num}).
Below is the board from your perspective:
  - Which words are still unrevealed (without color info),
  - Which words are revealed (with color).

{board}

Remember: Output exactly one guess word or 'end_turn'."""


class LLMSpymaster:
    def __init__(
        self,
        team: str,
        model: art.Model,
        temperature: float = 0.7,
        max_tokens: int = 20,
    ):
        self.team = team
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.conversation: List[Dict[str, str]] = [
            {"role": "system", "content": SPYMASTER_SYSTEM_PROMPT}
        ]
        self.conversation_choices: List[dict] = []

    def _board_summary(self, game_state):
        unrevealed_cards = game_state.get_unrevealed_cards()
        revealed_cards = game_state.get_revealed_cards()

        lines = []
        lines.append(f"Number of words left for blue: {game_state.words_left['blue']}")
        lines.append(f"Number of words left for red: {game_state.words_left['red']}")
        lines.append("UNREVEALED WORDS (and their colors):")
        # Group cards by color
        # Group cards by color and add them in a specific order
        for color in [
            self.team,
            "red" if self.team == "blue" else "blue",
            "neutral",
            "assassin",
        ]:
            for c in [c for c in unrevealed_cards if c.color == color]:
                lines.append(f"  {c.word} => {c.color}")
        lines.append("REVEALED WORDS (and their colors):")
        for c in revealed_cards:
            lines.append(f"  {c.word} => {c.color} (revealed)")

        return "\n".join(lines)

    def _parse_clue(self, text: str, gs: GameState) -> Tuple[str, int]:
        try:
            cw, num = (s.strip() for s in text.split(",", 1))
            num = int(num)
        except Exception:
            return "invalid", -1
        if not gs.follows_clue_rules(cw):
            return "invalid", -1
        return cw, num

    async def get_clue(self, gs: GameState, game_events: str) -> Tuple[str, int]:
        if game_events:
            self.conversation.append({"role": "user", "content": game_events})

        prompt = SPYMASTER_USER_PROMPT.format(
            team=self.team, board=self._board_summary(gs)
        )
        self.conversation.append({"role": "user", "content": prompt})

        resp = await call_llm(
            self.model,
            self.conversation,
            self.temperature,
            self.max_tokens,
        )
        self.conversation_choices.append(resp.choices[0])
        text = resp.choices[0].message.content.strip()  # type: ignore
        self.conversation.append({"role": "assistant", "content": text})

        return self._parse_clue(text, gs)


class LLMOperative:
    def __init__(
        self,
        team: str,
        model: art.Model,
        temperature: float = 0.7,
        max_tokens: int = 20,
    ):
        self.team = team
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.conversation: List[Dict[str, str]] = [
            {"role": "system", "content": OPERATIVE_SYSTEM_PROMPT}
        ]
        self.conversation_choices: List[dict] = []
        self.new_turn = True

    def _board_summary(self, game_state: GameState):
        """
        Build a summary from the operative's perspective:
          - unrevealed words (no color info)
          - revealed words (with color)
        """
        unrevealed = game_state.get_unrevealed_cards()
        revealed = game_state.get_revealed_cards()

        lines = []
        lines.append(f"Number of words left for blue: {game_state.words_left['blue']}")
        lines.append(f"Number of words left for red: {game_state.words_left['red']}")
        lines.append("UNREVEALED WORDS (no color info):")
        for c in unrevealed:
            lines.append(f"  {c.word}")
        lines.append("REVEALED WORDS (with color):")
        for c in revealed:
            lines.append(f"  {c.word}:{c.color}")
        return "\n".join(lines)

    async def guess_word(
        self,
        gs: GameState,
        clue_word: str,
        clue_number: int,
        game_events: str,
    ) -> str:
        if self.new_turn:
            self.new_turn = False
            if game_events:
                self.conversation.append({"role": "user", "content": game_events})
            prompt = OPERATIVE_USER_PROMPT.format(
                team=self.team,
                clue=clue_word,
                num=clue_number,
                board=self._board_summary(gs),
            )
            self.conversation.append({"role": "user", "content": prompt})

        resp = await call_llm(
            self.model,
            self.conversation,
            self.temperature,
            self.max_tokens,
        )
        self.conversation_choices.append(resp.choices[0])
        guess = resp.choices[0].message.content.strip()  # type: ignore
        self.conversation.append({"role": "assistant", "content": guess})
        return guess

    def start_new_turn(self):
        self.new_turn = True

    def add_guess_outcome(self, outcome: str):
        self.conversation.append({"role": "user", "content": outcome})

In [ ]:
# ───────────────────────  Gameplay  ──────────────────────────
async def run_game(
    gs: GameState,
    blue_spy: LLMSpymaster,
    blue_op: LLMOperative,
    red_spy: LLMSpymaster,
    red_op: LLMOperative,
) -> Tuple[str, str]:
    """
    Main loop – returns (winner, reason)
    """
    while True:
        # turn limit
        if gs.turn_count >= 50:
            gs.game_ended_in_turn_limit = True
            return "No winner", "Turn limit reached"

        gs.turn_count += 1

        # victory check
        winner, reason = gs.check_end_game()
        if winner:
            gs.log_event("game_engine", f"GAME OVER: {reason}")
            return winner, reason

        # who’s up?
        team = gs.current_team
        spy = blue_spy if team == "blue" else red_spy
        op = blue_op if team == "blue" else red_op
        spy_key = f"{team}_spymaster"
        op_key = f"{team}_operative"

        # ── Spymaster turn ──
        spy_events = gs.format_events_for_player(spy_key)
        clue_word, clue_num = await spy.get_clue(gs, spy_events)

        if clue_word == "invalid" or clue_num < 0:
            other = "red" if team == "blue" else "blue"
            reason = f"{team} gave invalid clue"
            gs.log_event("game_engine", f"GAME OVER: {other} wins – {reason}")
            gs.game_ended_in_error = {"team": team, "role": "spymaster"}
            return other, reason

        gs.log_event(spy_key, f"clue '{clue_word}', {clue_num}")

        # ── Operative guesses ──
        max_guesses = clue_num + 1
        guesses = 0
        op.start_new_turn()

        while guesses < max_guesses:
            # check win before guess
            winner, reason = gs.check_end_game()
            if winner:
                gs.log_event("game_engine", f"GAME OVER: {reason}")
                return winner, reason

            op_events = gs.format_events_for_player(op_key)
            guess = await op.guess_word(gs, clue_word, clue_num, op_events)

            if guess.lower() == "end_turn":
                if guesses == 0:
                    other = "red" if team == "blue" else "blue"
                    reason = f"{team} ended turn on the first guess"
                    gs.log_event("game_engine", f"GAME OVER: {other} wins – {reason}")
                    gs.game_ended_in_error = {"team": team, "role": "operative"}
                    return other, reason
                gs.log_event(op_key, "ended turn")
                break

            # validate guess
            card = next(
                (
                    c
                    for c in gs.get_unrevealed_cards()
                    if c.word.lower() == guess.lower()
                ),
                None,
            )
            if not card:
                other = "red" if team == "blue" else "blue"
                reason = f"{team} operative guessed invalid word '{guess}'"
                gs.log_event(op_key, reason)
                gs.log_event("game_engine", f"GAME OVER: {other} wins – {reason}")
                gs.game_ended_in_error = {"team": team, "role": "operative"}
                return other, reason

            color = gs.reveal_card(card)
            guesses += 1
            outcome = (
                "correct"
                if color == team
                else "assassin"
                if color == "assassin"
                else "other_team"
                if color in ("blue", "red")
                else "neutral"
            )
            gs.log_event(op_key, f"guessed '{guess}', outcome: {outcome}")

            # feedback for LLM memory
            op_feedback = create_operative_feedback(
                max_guesses, guesses, guess, color, outcome
            )
            op.add_guess_outcome(op_feedback)

            # post‑guess win check / turn handling
            if outcome == "correct":
                winner, reason = gs.check_end_game()
                if winner or guesses >= max_guesses:
                    break
                continue  # another guess
            elif outcome == "assassin":
                other = "red" if team == "blue" else "blue"
                gs.game_ended_in_assassin = True
                return other, f"{team} revealed the assassin"
            else:
                break  # neutral or other team ends turn

        gs.switch_turn()

In [ ]:
# ────────────────────  5. RL / reward helpers  ────────────────────────────
def create_mc(player: LLMSpymaster | LLMOperative) -> List[dict]:
    """
    Merge messages & choices into a single list for art.
    """
    mc = []
    choice_i = 0
    for msg in player.conversation:
        if msg["role"] == "assistant":
            mc.append(player.conversation_choices[choice_i])
            choice_i += 1
        else:
            mc.append(msg)
    return mc


def create_margin_reward(
    player: LLMSpymaster | LLMOperative,
    role_key: str,
    gs: GameState,
    winning_color: str,
    reason: str,
) -> float:
    num_start_words = 9 if player.team == "blue" else 8
    reward = 0.0

    if gs.game_ended_in_turn_limit:
        reward += MARGIN_REWARD_DICT["turn_limit"]
        return reward
    if (
        gs.game_ended_in_error
        and role_key
        == f"{gs.game_ended_in_error['team']}_{gs.game_ended_in_error['role']}"
    ):
        reward += MARGIN_REWARD_DICT["error"]

    if gs.game_ended_in_assassin and player.team != winning_color:
        reward += MARGIN_REWARD_DICT["assassin"]

    # proportion of own words guessed
    reward += (
        (num_start_words - gs.words_left[player.team])
        / num_start_words
        * MARGIN_REWARD_DICT["num_words_multiplier"]
    )

    # margin
    loser = "red" if winning_color == "blue" else "blue"
    margin = gs.words_left[loser]
    reward += (
        margin
        * MARGIN_REWARD_DICT["margin_multiplier"]
        * (1 if player.team == winning_color else -1)
    )
    return reward

# Defining a rollout

<a name="Rollout"></a>

In each training step, we start with NUM_GAMES_PER_STEP different Codenames boards (let's say 10).

NUM_GAMES_PER_STEP = 10

For every board, the model plays 20 complete self‑play games (rollouts).

NUM_ROLLOUTS_PER_GAME = 20

Each rollout produces four separate records of experience—one for every role in the game—so a single board yields 80 trajectories in total.

trajectories_per_rollout = 4 (one each for red spymaster, red operative, blue spymaster, blue operative)

trajectories_per_board = NUM_ROLLOUTS_PER_GAME _ trajectories_per_rollout = 20 _ 4 = 80

All trajectories from the same board+color+role are grouped together into a trajectory group.

trajectory_groups_per_board = trajectories_per_board / NUM_ROLLOUTS_PER_GAME = 80 / 20 = 4

Across the NUM_GAMES_PER_STEP (10) boards, this gives us NUM_GAMES_PER_STEP _ trajectory_groups_per_board = 10 _ 4 = 40 trajectory groups that ART uses for training in that step.


In [ ]:
# ───────────────────────  6. Rollout logic  ───────────────────────────────
async def run_single_rollout(
    game_state: GameState,
    blue_model: art.Model,
    red_model: art.Model,
    rollout_id: int,
    step: int,
) -> Dict[str, Dict[str, art.Trajectory]] | Exception:
    """
    Play one rollout, i.e. one game, return trajectories for every player.
    For each rollout, we return four trajectories: red spymaster, red operative, blue spymaster, blue operative.
    We conduct the game, caluclate the reward for each trajectory, report to OpenPipe, and return the trajectories.
    """
    player_classes = {"spymaster": LLMSpymaster, "operative": LLMOperative}
    player_dict: Dict[str, Dict[str, LLMSpymaster | LLMOperative]] = {
        "blue": {
            role: player_classes[role](team="blue", model=blue_model)
            for role in ("spymaster", "operative")
        },
        "red": {
            role: player_classes[role](team="red", model=red_model)
            for role in ("spymaster", "operative")
        },
    }

    try:
        winner, reason = await run_game(
            gs=game_state,
            blue_spy=player_dict["blue"]["spymaster"],  # type: ignore
            blue_op=player_dict["blue"]["operative"],  # type: ignore
            red_spy=player_dict["red"]["spymaster"],  # type: ignore
            red_op=player_dict["red"]["operative"],  # type: ignore
        )
    except Exception as e:
        return e

    # ── build art trajectories ──
    game_metrics = {
        "ended_in_error": 1 if game_state.game_ended_in_error else 0,
        "ended_in_assassin": 1 if game_state.game_ended_in_assassin else 0,
        "ended_completely": 0
        if (game_state.game_ended_in_error or game_state.game_ended_in_assassin)
        else 1,
        "total_turns": game_state.turn_count,
        "total_words_left": game_state.words_left["blue"]
        + game_state.words_left["red"],
        "winning_color": 1 if winner == "blue" else 0,
    }

    op_client = AsyncOpenPipe()
    trajectory_dict: Dict[str, Dict[str, art.Trajectory]] = {}

    for team in ("blue", "red"):
        for role in ("spymaster", "operative"):
            player = player_dict[team][role]
            role_key = f"{team}_{role}"
            # finish conversation with final message
            events = game_state.format_events_for_player(role_key)
            if events:
                player.conversation.append({"role": "user", "content": events})
            player.conversation.append(
                {
                    "role": "user",
                    "content": f"Game over. {winner} won. Reason: {reason}",
                }
            )

            reward = create_margin_reward(player, role_key, game_state, winner, reason)
            mc = create_mc(player)
            traj = art.Trajectory(
                messages_and_choices=mc,
                reward=reward,
                metrics={"reward": reward, **game_metrics},
            )
            trajectory_dict.setdefault(team, {})[role] = traj

            # report to OpenPipe for observability (best‑effort)
            try:
                await op_client.report(
                    req_payload={
                        "model": f"{player.model.name}_{team}",
                        "messages": player.conversation,
                        "metadata": {
                            "game_id": game_state.game_id,
                            "rollout_id": rollout_id,
                            "step": step,
                            "role": role,
                            "team": team,
                            "num_turns": game_state.turn_count,
                            "blue_words_left": game_state.words_left["blue"],
                            "red_words_left": game_state.words_left["red"],
                            "winning_color": winner,
                            "game_end_reason": reason[:1000],
                            "reward": reward,
                        },
                    },
                    resp_payload={
                        "id": f"chatcmpl-{uuid.uuid4()}",
                        "object": "chat.completion",
                        "created": int(time.time()),
                        "model": f"{player.model.name}_{team}",
                        "choices": [
                            {
                                "index": 0,
                                "message": {"role": "assistant", "content": "dummy"},
                                "finish_reason": "stop",
                            }
                        ],
                        "usage": {
                            "prompt_tokens": 0,
                            "completion_tokens": 0,
                            "total_tokens": 0,
                        },
                    },
                    status_code=200,
                )
            except Exception as e:
                if DEBUG:
                    print(f"OpenPipe report failed: {e}")

    return trajectory_dict


async def conduct_games(
    blue_model: art.Model,
    red_model: art.Model,
    num_rollouts_per_game: int,
    game_id: int,
    step: int,
) -> List[art.TrajectoryGroup]:
    """
    Play *num_rollouts_per_game* games concurrently for a fixed board.
    The fixed board provides the same starting state for each rollout.
    """
    base_state = create_game(game_id=game_id)

    rollout_results = await asyncio.gather(
        *[
            run_single_rollout(
                game_state=copy.deepcopy(base_state),
                blue_model=blue_model,
                red_model=red_model,
                rollout_id=rid,
                step=step,
            )
            for rid in range(num_rollouts_per_game)
        ]
    )

    groups: Dict[str, List[art.Trajectory]] = {}
    for res in rollout_results:
        if isinstance(res, Exception):
            continue
        for team in ("blue", "red"):
            for role in ("spymaster", "operative"):
                groups.setdefault(f"{team}_{role}", []).append(res[team][role])

    return [art.TrajectoryGroup(trajs) for trajs in groups.values()]


async def _run_single_validation_game(
    game_id: int,
    my_model: art.Model,
    benchmark_model: art.Model,
    step: int = 0,
) -> art.Trajectory | Exception:
    """
    Creates a single game of Codenames, alternating your model and the benchmark model
    (e.g. GPT-4o) between Blue and Red. Returns a list of your model’s Trajectories only, with the reward
    for each set to 1 if your model won, and 0 otherwise.
    """

    # Create a fresh game state
    game_state = create_game(game_id=game_id)

    # Alternate which side your model takes
    if game_id % 2 == 0:
        my_team, _other_team = "blue", "red"
        blue_model, red_model = my_model, benchmark_model
    else:
        my_team, _other_team = "red", "blue"
        blue_model, red_model = benchmark_model, my_model

    # Run a single rollout (one game) using your existing code
    trajectory_dict = await run_single_rollout(
        game_state=copy.deepcopy(game_state),
        blue_model=blue_model,
        red_model=red_model,
        rollout_id=game_id,
        step=step,
    )

    if isinstance(trajectory_dict, Exception):
        return trajectory_dict

    winning_color = (
        "blue"
        if trajectory_dict["blue"]["spymaster"].metrics.get("winning_color", "") == 1
        else "red"
    )

    my_traj = trajectory_dict[my_team]["spymaster"]
    my_traj_reward = 1.0 if my_team == winning_color else 0.0

    # Overwrite the reward in the trajectory
    my_traj.reward = my_traj_reward
    my_traj.metrics["win_rate"] = my_traj_reward
    my_traj.metrics["reward"] = my_traj_reward

    return my_traj


async def run_validation_games(
    my_model: art.Model,
    benchmark_model: art.Model,
    num_validation_games: int = 50,
    step: int = 0,
) -> List[art.Trajectory]:
    """
    Creates `num_validation_games` games of Codenames, concurrently running them via asyncio.gather,
    alternating your model vs. benchmark model (e.g. GPT‑4o) between Blue and Red.

    Returns a flat list of your model’s trajectories. Each trajectory has a reward of 1 if your
    model’s team won, and 0 if it lost.
    """
    tasks = [
        _run_single_validation_game(
            game_id=gid,
            my_model=my_model,
            benchmark_model=benchmark_model,
            step=step,
        )
        for gid in range(num_validation_games)
    ]

    # Run all validation games concurrently
    result_trajectories = await tqdm_asyncio.gather(
        *tasks, desc="Running validation games", total=num_validation_games
    )

    return [t for t in result_trajectories if not isinstance(t, Exception)]

# Model and Training Loop

<a name="Loop"></a>

First, we create a trainable model which gives ART a spec of the model we want to train.
Next, we create a backend, which could be local or point to a remote server which has art installed.
We then register the model with the backend, letting the backend know where we will be training the model.
You then have an openai client for your model, which will automatically point to the latest version of the model under training.

For the training loop, we conduct training games and every 5 steps, we conduct validation games against val benchmark model.
For every training step, we gather the trajectory groups from each starting position, flatten into groups, and then train the model on these groups.

We provide additional benchmaking code to see how the model performs over time when compared to off the shelf models like gpt-4.1-nano and gpt-4.1-mini.


In [ ]:
# ───────────────────────────  7. Creating the model  ───────────────────────────
from art.local import LocalBackend

model = art.TrainableModel(
    name=MODEL_NAME,
    project="codenames-rl",
    base_model=BASE_MODEL_NAME,
)

benchmark_model = art.Model(
    name=VAL_BENCHMARK_MODEL,
    project="codenames-rl",
    inference_model_name=VAL_BENCHMARK_MODEL,
    inference_api_key=os.getenv("OPENAI_API_KEY"),
    inference_base_url="https://api.openai.com/v1",
)

backend = LocalBackend(path="./.art")

await model.register(backend)
await benchmark_model.register(backend)

In [ ]:
# ───────────────────────────  8. Training loop  ───────────────────────────
for step in range(await model.get_step(), NUM_STEPS):
    # Our model under training plays against itself and learns from the rewards for each trajectory
    nested_groups = await tqdm_asyncio.gather(
        *[
            conduct_games(
                blue_model=model,
                red_model=model,
                num_rollouts_per_game=NUM_ROLLOUTS_PER_GAME,
                game_id=gid,
                step=step,
            )
            for gid in range(NUM_GAMES_PER_STEP)
        ],
        total=NUM_GAMES_PER_STEP,
        desc=f"step {step}",
    )

    if step % 5 == 0:
        # We run validation games to evaluate the performance of our model against the benchmark model. Here, the reward is 1 if our model wins, and 0 otherwise.
        val_groups = await run_validation_games(
            my_model=model,
            benchmark_model=benchmark_model,
            num_validation_games=NUM_VALIDATION_GAMES,
        )
        await model.log(val_groups)

    flat_groups: List[art.TrajectoryGroup] = [g for sub in nested_groups for g in sub]
    await model.train(
        trajectory_groups=flat_groups,
        config=art.TrainConfig(learning_rate=LEARNING_RATE, beta=BETA),
    )
    await model.delete_checkpoints()
    print(f"✓ finished step {step}")

In [ ]:
# ───────────────────────────  9. Benchmark model  ───────────────────────────
# We will benchmark our model by comparing the win rates against the benchmark model with that of gpt-4.1-nano and gpt-4.1-mini.
gpt_41_nano = art.Model(
    name="gpt-4.1-nano",
    project="codenames-rl",
    inference_model_name="gpt-4.1-nano",
    inference_api_key=os.getenv("OPENAI_API_KEY"),
    inference_base_url="https://api.openai.com/v1",
)
gpt_41_mini = art.Model(
    name="gpt-4.1-mini",
    project="codenames-rl",
    inference_model_name="gpt-4.1-mini",
    inference_api_key=os.getenv("OPENAI_API_KEY"),
    inference_base_url="https://api.openai.com/v1",
)

gpt_models = [gpt_41_nano, gpt_41_mini]

for gpt_model in gpt_models:
    await gpt_model.register(backend)
    val_groups = await run_validation_games(
        my_model=gpt_model,
        benchmark_model=benchmark_model,
        num_validation_games=NUM_VALIDATION_GAMES,
    )
    await gpt_model.log(val_groups)

In [ ]:
# ───────────────────────────  10. Plotting Utilities  ───────────────────────────
import os

from art.utils.benchmarking.load_trajectories import load_trajectories
from art.utils.benchmarking.charts import training_progress_chart
from art.utils.benchmarking.types import BenchmarkModelKey

df = await load_trajectories(
    project_name="codenames-rl",
    models=[MODEL_NAME, "gpt-4.1-nano", "gpt-4.1-mini"],
    art_path="./.art",
)

In [ ]:
# ───────────────────────────  11. Plotting Win Rate over Time  ───────────────────────────
line_graph = training_progress_chart(
    df,
    "win_rate",
    models=[
        BenchmarkModelKey(MODEL_NAME, MODEL_NAME, "val"),
        BenchmarkModelKey("gpt-4.1-nano", "GPT-4.1-nano", "val"),
        BenchmarkModelKey("gpt-4.1-mini", "GPT-4.1-mini", "val"),
    ],
    title="Win Rate over Time",
    y_label="Win Rate",
)

line_graph.savefig("win_rate_over_time.png")

<div class="align-center">
<a href="https://github.com/openpipe/art"><img src="https://github.com/openpipe/art/raw/main/assets/ART_pill.png" height="50"></a>
<a href="https://discord.com/invite/dnseNZuQ"><img src="https://github.com/openpipe/art/raw/main/assets/Discord_pill.png" height="50"></a>
<a href="https://openpipe.ai/blog/art-e-mail-agent"><img src="https://github.com/openpipe/art/raw/main/assets/ART_E_pill.png" height="50"></a>

Questions? Join the Discord and ask away! For feature requests or to leave a star, visit our [GitHub](https://github.com/openpipe/art).

</div>
